In [1]:
import os

In [2]:
!pwd

/home/jovyan/notebooks/executions


In [3]:
import sys
sys.path.append('/home/jovyan/')

In [4]:
from experiments.reporting.report_repository import ReportRepository
from experiments.slack_client import SlackNotifier
from neat.neat_logger import get_neat_logger
from neat.population_engine import EvolutionEngine
from neat.reporting.reports_pyneat import EvolutionReport
from neat.utils import timeit
import math
import random
from itertools import count
import numpy as np

from experiments.logger import logger
from experiments.slack_client import Notifier
from neat.configuration import get_configuration
from neat.evaluation.evaluation_engine import EvaluationStochasticEngine
from neat.evolution_operators.crossover import Crossover
from neat.evolution_operators.mutation import Mutation
from neat.genome import Genome
from neat.reporting.reports_pyneat import EvolutionReport
from neat.species import SpeciationEngine
from neat.stagnation import Stagnation
from neat.population_engine import PopulationEngine
from neat.utils import timeit
import neat
import tests
from config_files.configuration_utils import create_configuration

In [5]:
# dir(neat)

In [6]:
LOGS_PATH = f'{os.getcwd()}/'
logger = get_neat_logger(path=LOGS_PATH)

### Configuration

In [7]:
config_file = 'mnist_binary'
config = create_configuration(filename=f'/{config_file}.json')

In [13]:
# TODO: better mechanism for override
config.pop_size = 20
# config.n_samples = 50
#
config.n_generations = 300
config.node_add_prob = 0.5
config.n_output = 2
config.n_processes = 2


ALGORITHM_VERSION = 'bayes-neat'
DATASET = 'mnist_binary'
# CORRELATION_ID = 'parameters_grid'
CORRELATION_ID = 'solve_binary'

In [14]:
report_repository = ReportRepository.create(project='neuro-evolution', logs_path=LOGS_PATH)
notifier = SlackNotifier.create(channel='batch-jobs')

In [15]:
report = EvolutionReport(report_repository=report_repository,
                         algorithm_version=ALGORITHM_VERSION,
                         dataset=DATASET,
                         correlation_id=CORRELATION_ID)

In [16]:
population_engine = PopulationEngine(stagnation_engine=Stagnation())
speciation_engine = SpeciationEngine()
evaluation_engine = EvaluationStochasticEngine()

GETTING DATASET
GETTING DATASET


In [17]:
evaluation_engine.n_processes

2

### Initialize

In [18]:
end_condition = 'normal'
# try:
# initialize population
population = population_engine.initialize_population()
speciation_engine.speciate(population, generation=0)

population = evaluation_engine.evaluate(population=population)

# report
report.report_new_generation(generation=0,
                                  population=population,
                                  species=speciation_engine.species)

2020-03-19 18:15:03,708 - INFO - Generation 0. Best fitness: -198.308. N-Parameters Best: 8
2020-03-19 18:15:03,709 - INFO -                          Mean fitness: -199.773. Mean N-Parameters: 8.0
2020-03-19 18:15:03,710 - INFO -     New best individual (20) found with fitness -198.308
GETTING DATASET
Confusion Matrix:
[[48435 49565]
 [56598 56902]]
Accuracy: 49.8047281323877 %


In [22]:
def run_generation(generation, population):
    
    # create new generation's population
    population = population_engine.reproduce(species=speciation_engine.species,
                                             pop_size=population_engine.pop_size,
                                             generation=generation)
    # create new species based on new population
    speciation_engine.speciate(population, generation=generation)

    # evaluate
    population = evaluation_engine.evaluate(population=population)

    # generation report
    report.report_new_generation(generation=generation,
                                 population=population,
                                 species=speciation_engine.species)
    generation += 1
    return population

In [23]:
generation = 1

In [27]:
generation += 1
population = run_generation(generation, population)

2020-03-20 09:39:33,083 - INFO - Generation 3. Best fitness: -0.956. N-Parameters Best: 24
2020-03-20 09:39:33,083 - INFO -                          Mean fitness: -21.167. Mean N-Parameters: 22.8


### Check evaluation results

In [31]:
from neat.evaluation.evaluate_simple import evaluate_genome
from neat.evaluation.utils import get_dataset
from neat.loss.vi_loss import get_loss, get_beta

In [29]:
testing = False

In [33]:
dataset = get_dataset(config.dataset, testing=testing)
loss = get_loss(problem_type=config.problem_type)

GETTING DATASET
